<a href="https://colab.research.google.com/github/Divide-N-Conquer/EmotionalChatbot/blob/master/BERT%20(First%20Dataset)%20v3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
import pandas as pd
url = 'https://raw.githubusercontent.com/Divide-N-Conquer/EmotionalChatbot/master/CBET.csv'
dataset = pd.read_csv(url)

dataset.insert(2,'label',-1)
dataset.head(5)
dataset.shape
# filtered = dataset[dataset["love"] != 1.0]
# filtered = filtered[dataset["thankfulness"] != 1.0]
# filtered = filtered[dataset["guilt"] != 1.0]
# filtered.shape
# filtered.head(5)
filtered = dataset

filtered.loc[(filtered["anger"] > 0) & (filtered['label'] == -1), 'label'] = 5
filtered.loc[(filtered["fear"] > 0) & (filtered['label'] == -1), 'label'] = 3
filtered.loc[(filtered["joy"] > 0) & (filtered['label'] == -1), 'label'] = 0
filtered.loc[(filtered["sadness"] > 0) & (filtered['label'] == -1), 'label'] = 1
filtered.loc[(filtered["surprise"] > 0) & (filtered['label'] == -1), 'label'] = 4
filtered.loc[(filtered["disgust"] > 0) & (filtered['label'] == -1), 'label'] = 2
filtered.loc[(filtered["love"] > 0) & (filtered['label'] == -1), 'label'] = 6
filtered.loc[(filtered["thankfulness"] > 0) & (filtered['label'] == -1), 'label'] = 7
filtered.loc[(filtered["guilt"] > 0) & (filtered['label'] == -1), 'label'] = 8


filtered = filtered[["text","label"]]
filtered.head(5)

# Get the lists of sentences and their labels.
# sentences = filtered.text.values
# labels = filtered.label.values
# filtered.shape

,text,label
0,In hours i have to drive to applegate four hou...,5
1,When there s no more seats available in a rest...,5
2,PTGforza when people copy your hard earned wor...,5
3,I was just researching that online THIS IS WRO...,5
4,photo Angry #Headmaster milton grimm #MiltonGr...,5


In [4]:
train_size = int(0.7 * len(filtered))
val_size = len(filtered) - train_size

anger = filtered[filtered["label"] == 5]
fear = filtered[filtered["label"] == 3]
joy = filtered[filtered["label"] == 0]
sadness = filtered[filtered["label"] == 3]
surprise = filtered[filtered["label"] == 4]
disgust = filtered[filtered["label"] == 2]
love = filtered[filtered["label"] == 6]
thankfulness = filtered[filtered["label"] == 7]
guilt = filtered[filtered["label"] == 8]

anger_copy = anger.copy()
fear_copy = fear.copy()
joy_copy = joy.copy()
sadness_copy = sadness.copy()
surprise_copy = surprise.copy()
disgust_copy = disgust.copy()
love_copy = love.copy()
thankfulness_copy = thankfulness.copy()
guilt_copy = guilt.copy()

train_set_1 = anger_copy.sample(frac=0.7, random_state=0)
test_set_1 = anger_copy.drop(train_set_1.index)
train_set_2 = fear_copy.sample(frac=0.7, random_state=0)
test_set_2 = fear_copy.drop(train_set_2.index)
train_set_3 = joy_copy.sample(frac=0.7, random_state=0)
test_set_3 = joy_copy.drop(train_set_3.index)
train_set_4 = sadness_copy.sample(frac=0.7, random_state=0)
test_set_4 = sadness_copy.drop(train_set_4.index)
train_set_5 = surprise_copy.sample(frac=0.7, random_state=0)
test_set_5 = surprise_copy.drop(train_set_5.index)
train_set_6 = disgust_copy.sample(frac=0.7, random_state=0)
test_set_6 = disgust_copy.drop(train_set_6.index)
train_set_7 = love_copy.sample(frac=0.7, random_state=0)
test_set_7 = love_copy.drop(train_set_7.index)
train_set_8 = thankfulness_copy.sample(frac=0.7, random_state=0)
test_set_8 = thankfulness_copy.drop(train_set_8.index)
train_set_9 = guilt_copy.sample(frac=0.7, random_state=0)
test_set_9 = guilt_copy.drop(train_set_9.index)

frames = [train_set_1,train_set_2,train_set_3,train_set_4,train_set_5,train_set_6,train_set_7,train_set_8,train_set_9]
frames1 = [test_set_1,test_set_2,test_set_3,test_set_4,test_set_5,test_set_6,test_set_7,test_set_8,test_set_9]
train_dataset = pd.concat(frames)
val_dataset = pd.concat(frames1)
print(train_dataset.shape)
print(val_dataset.shape)

sentences = train_dataset.text.values
labels = train_dataset.label.values

sentences1 = val_dataset.text.values

labels1 = val_dataset.label.values

(56923, 2)
(24397, 2)


In [5]:
import tensorflow as tf

# Get the GPU device name.
device_name = tf.test.gpu_device_name()

# The device name should look like the following:
if device_name == '/device:GPU:0':
    print('Found GPU at: {}'.format(device_name))
else:
    raise SystemError('GPU device not found')

Found GPU at: /device:GPU:0


In [6]:
import torch

# If there's a GPU available...
if torch.cuda.is_available():    

    # Tell PyTorch to use the GPU.    
    device = torch.device("cuda")

    print('There are %d GPU(s) available.' % torch.cuda.device_count())

    print('We will use the GPU:', torch.cuda.get_device_name(0))

# If not...
else:
    print('No GPU available, using the CPU instead.')
    device = torch.device("cpu")

There are 1 GPU(s) available.
We will use the GPU: Tesla T4


In [7]:
!pip install transformers

     |████████████████████████████████| 778kB 4.6MB/s 
     |████████████████████████████████| 3.0MB 23.2MB/s 
     |████████████████████████████████| 890kB 52.7MB/s 
     |████████████████████████████████| 1.1MB 52.1MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp36-none-any.whl size=893260 sha256=175d13886ee355fd9ba46247031ad39eb3d06d78d0bd8017d5f285e8dfa446e9
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sacremoses


In [8]:
from transformers import BertTokenizer

# Load the BERT tokenizer.
print('Loading BERT tokenizer...')
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased', do_lower_case=True)

Loading BERT tokenizer...


In [9]:
max_len = 0

# For every sentence...
for sent in sentences:

    # Tokenize the text and add `[CLS]` and `[SEP]` tokens.
    input_ids = tokenizer.encode(sent, add_special_tokens=True)

    # Update the maximum sentence length.
    max_len = max(max_len, len(input_ids))

print('Max sentence length: ', max_len)

Max sentence length:  121


In [11]:
# Tokenize all of the sentences and map the tokens to thier word IDs.
input_ids = []
attention_masks = []
punctuation = '!"#$%&()*+-/:;<=>?@[\\]^_`{|}~'
# For every sentence...
for sent in sentences:
    sent = ''.join(ch for ch in sent if ch not in set(punctuation))
    sent = ''.join(ch for ch in sent if len(ch) < 18)
    sent = sent.lower()
    sent = ' '.join(sent.split())
    # `encode_plus` will:
    #   (1) Tokenize the sentence.
    #   (2) Prepend the `[CLS]` token to the start.
    #   (3) Append the `[SEP]` token to the end.
    #   (4) Map tokens to their IDs.
    #   (5) Pad or truncate the sentence to `max_length`
    #   (6) Create attention masks for [PAD] tokens.
    encoded_dict = tokenizer.encode_plus(
                        sent,                      # Sentence to encode.
                        add_special_tokens = True, # Add '[CLS]' and '[SEP]'
                        max_length = 128,           # Pad & truncate all sentences.
                        pad_to_max_length = True,
                        return_attention_mask = True,   # Construct attn. masks.
                        return_tensors = 'pt',     # Return pytorch tensors.
                        truncation = True,
                   )
    
    # Add the encoded sentence to the list.    
    input_ids.append(encoded_dict['input_ids'])
    
    # And its attention mask (simply differentiates padding from non-padding).
    attention_masks.append(encoded_dict['attention_mask'])

# Convert the lists into tensors.
input_ids = torch.cat(input_ids, dim=0)
attention_masks = torch.cat(attention_masks, dim=0)
labels = torch.tensor(labels)

# Print sentence 0, now as a list of IDs.
print('Original: ', sentences[0])
print('Token IDs:', input_ids[0])

noooo wcsx you were my last hope of hearing city of the angels arrrrrrrggggg
kash zama or a ganic chemistry teacher mar she omg i am so evil i take that back i just wish che de aga magz a allah ma ta rakee
streaming some aw in a couple minutes come by and say hi d cod advanced warfare codadvancedwarfare
jiang homes hi jianghomeshi your followers have stopped believing lost faith and see you for what you are shame
win tickets markus schulz at club space miami rave eat sleep rave repeat eatsleepraverepeat plur techno trance edm so be sobe nye
lilmaa dejloaf i love her n orlando jealous in the north
pricks in supermarkets bim bling about on their phone and then moaning when they get a trolley to the back get out the fc king way
user money can t buy you love but it sure can buy you an election there goes fine arts theregoesfinearts goodbye education goodbyeeducation
last nights fortune is so relevant damn chinese food chinesefood fortune sorrow
you fkn hippo biatch
wylddandelyon i should b

KeyboardInterrupt: ignored

In [ ]:
# Tokenize all of the sentences and map the tokens to thier word IDs.
input_ids1 = []
attention_masks1 = []
punctuation = '!"#$%&()*+-/:;<=>?@[\\]^_`{|}~'
# For every sentence...
for sent in sentences1:
    sent = ''.join(ch for ch in sent if ch not in set(punctuation))
    sent = ''.join(ch for ch in sent if len(ch) < 18)
    sent = sent.lower()
    sent = ' '.join(sent.split())
    # `encode_plus` will:
    #   (1) Tokenize the sentence.
    #   (2) Prepend the `[CLS]` token to the start.
    #   (3) Append the `[SEP]` token to the end.
    #   (4) Map tokens to their IDs.
    #   (5) Pad or truncate the sentence to `max_length`
    #   (6) Create attention masks for [PAD] tokens.
    encoded_dict1 = tokenizer.encode_plus(
                        sent,                      # Sentence to encode.
                        add_special_tokens = True, # Add '[CLS]' and '[SEP]'
                        max_length = 128,           # Pad & truncate all sentences.
                        pad_to_max_length = True,
                        return_attention_mask = True,   # Construct attn. masks.
                        return_tensors = 'pt',     # Return pytorch tensors.
                        truncation = True,
                   )
    
    # Add the encoded sentence to the list.    
    input_ids1.append(encoded_dict1['input_ids'])
    
    # And its attention mask (simply differentiates padding from non-padding).
    attention_masks1.append(encoded_dict1['attention_mask'])

# Convert the lists into tensors.
input_ids1 = torch.cat(input_ids1, dim=0)
attention_masks1 = torch.cat(attention_masks1, dim=0)
labels1 = torch.tensor(labels1)

# Print sentence 0, now as a list of IDs.
print('Original: ', sentences1[0])
print('Token IDs:', input_ids1[0])

Original:  In hours i have to drive to applegate four hours up north But its also Friday so
Token IDs: tensor([ 101, 1999, 2847, 1045, 2031, 2000, 3298, 2000, 6207, 5867, 2176, 2847,
        2039, 2167, 2021, 2049, 2036, 5958, 2061,  102,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0])


In [ ]:
from torch.utils.data import TensorDataset, random_split

# Combine the training inputs into a TensorDataset.
# dataset = TensorDataset(input_ids, attention_masks, labels)
train_dataset = TensorDataset(input_ids, attention_masks, labels)
val_dataset = TensorDataset(input_ids1, attention_masks1, labels1)


# Create a 90-10 train-validation split.

# Calculate the number of samples to include in each set.
# train_size = int(0.7 * len(dataset))
# val_size = len(dataset) - train_size

# Divide the dataset by randomly selecting samples.
# train_dataset, val_dataset = random_split(dataset, [train_size, val_size])

# print('{:>5,} training samples'.format(train_size))
# print('{:>5,} validation samples'.format(val_size))

In [ ]:
from torch.utils.data import DataLoader, RandomSampler, SequentialSampler

# The DataLoader needs to know our batch size for training, so we specify it 
# here. For fine-tuning BERT on a specific task, the authors recommend a batch 
# size of 16 or 32.
batch_size = 32

# Create the DataLoaders for our training and validation sets.
# We'll take training samples in random order. 
train_dataloader = DataLoader(
            train_dataset,  # The training samples.
            sampler = RandomSampler(train_dataset), # Select batches randomly
            batch_size = batch_size # Trains with this batch size.
        )

# For validation the order doesn't matter, so we'll just read them sequentially.
validation_dataloader = DataLoader(
            val_dataset, # The validation samples.
            sampler = SequentialSampler(val_dataset), # Pull out batches sequentially.
            batch_size = batch_size # Evaluate with this batch size.
        )

In [ ]:
from transformers import BertForSequenceClassification, AdamW, BertConfig

# Load BertForSequenceClassification, the pretrained BERT model with a single 
# linear classification layer on top. 
model = BertForSequenceClassification.from_pretrained(
    "bert-base-uncased", # Use the 12-layer BERT model, with an uncased vocab.
    num_labels = 9, # The number of output labels--2 for binary classification.
                    # You can increase this for multi-class tasks.   
    output_attentions = False, # Whether the model returns attentions weights.
    output_hidden_states = False, # Whether the model returns all hidden-states.
)

# Tell pytorch to run this model on the GPU.
model.cuda()

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPretraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, element

In [ ]:
# Get all of the model's parameters as a list of tuples.
params = list(model.named_parameters())

print('The BERT model has {:} different named parameters.\n'.format(len(params)))

print('==== Embedding Layer ====\n')

for p in params[0:5]:
    print("{:<55} {:>12}".format(p[0], str(tuple(p[1].size()))))

print('\n==== First Transformer ====\n')

for p in params[5:21]:
    print("{:<55} {:>12}".format(p[0], str(tuple(p[1].size()))))

print('\n==== Output Layer ====\n')

for p in params[-4:]:
    print("{:<55} {:>12}".format(p[0], str(tuple(p[1].size()))))

The BERT model has 201 different named parameters.

==== Embedding Layer ====

bert.embeddings.word_embeddings.weight                  (30522, 768)
bert.embeddings.position_embeddings.weight                (512, 768)
bert.embeddings.token_type_embeddings.weight                (2, 768)
bert.embeddings.LayerNorm.weight                              (768,)
bert.embeddings.LayerNorm.bias                                (768,)

==== First Transformer ====

bert.encoder.layer.0.attention.self.query.weight          (768, 768)
bert.encoder.layer.0.attention.self.query.bias                (768,)
bert.encoder.layer.0.attention.self.key.weight            (768, 768)
bert.encoder.layer.0.attention.self.key.bias                  (768,)
bert.encoder.layer.0.attention.self.value.weight          (768, 768)
bert.encoder.layer.0.attention.self.value.bias                (768,)
bert.encoder.layer.0.attention.output.dense.weight        (768, 768)
bert.encoder.layer.0.attention.output.dense.bias              (

In [ ]:
# Note: AdamW is a class from the huggingface library (as opposed to pytorch) 
# I believe the 'W' stands for 'Weight Decay fix"
optimizer = AdamW(model.parameters(),
                  lr = 2e-5, # args.learning_rate - default is 5e-5, our notebook had 2e-5
                  eps = 1e-8 # args.adam_epsilon  - default is 1e-8.
                )

In [ ]:
from transformers import get_linear_schedule_with_warmup

# Number of training epochs. The BERT authors recommend between 2 and 4. 
# We chose to run for 4, but we'll see later that this may be over-fitting the
# training data.
epochs = 2

# Total number of training steps is [number of batches] x [number of epochs]. 
# (Note that this is not the same as the number of training samples).
total_steps = len(train_dataloader) * epochs

# Create the learning rate scheduler.
scheduler = get_linear_schedule_with_warmup(optimizer, 
                                            num_warmup_steps = 0, # Default value in run_glue.py
                                            num_training_steps = total_steps)

In [ ]:
import numpy as np

# Function to calculate the accuracy of our predictions vs labels
def flat_accuracy(preds, labels):
    pred_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()
    return np.sum(pred_flat == labels_flat) / len(labels_flat)

In [ ]:
import time
import datetime

def format_time(elapsed):
    '''
    Takes a time in seconds and returns a string hh:mm:ss
    '''
    # Round to the nearest second.
    elapsed_rounded = int(round((elapsed)))
    
    # Format as hh:mm:ss
    return str(datetime.timedelta(seconds=elapsed_rounded))

In [ ]:
import random
import numpy as np

# This training code is based on the `run_glue.py` script here:
# https://github.com/huggingface/transformers/blob/5bfcd0485ece086ebcbed2d008813037968a9e58/examples/run_glue.py#L128

# Set the seed value all over the place to make this reproducible.
seed_val = 42

random.seed(seed_val)
np.random.seed(seed_val)
torch.manual_seed(seed_val)
torch.cuda.manual_seed_all(seed_val)

# We'll store a number of quantities such as training and validation loss, 
# validation accuracy, and timings.
training_stats = []

# Measure the total training time for the whole run.
total_t0 = time.time()

# For each epoch...
for epoch_i in range(0, epochs):
    
    # ========================================
    #               Training
    # ========================================
    
    # Perform one full pass over the training set.

    print("")
    print('======== Epoch {:} / {:} ========'.format(epoch_i + 1, epochs))
    print('Training...')

    # Measure how long the training epoch takes.
    t0 = time.time()

    # Reset the total loss for this epoch.
    total_train_loss = 0

    # Put the model into training mode. Don't be mislead--the call to 
    # `train` just changes the *mode*, it doesn't *perform* the training.
    # `dropout` and `batchnorm` layers behave differently during training
    # vs. test (source: https://stackoverflow.com/questions/51433378/what-does-model-train-do-in-pytorch)
    model.train()
    

    # For each batch of training data...
    for step, batch in enumerate(train_dataloader):
        # Progress update every 40 batches.
        if step % 40 == 0 and not step == 0:
            # Calculate elapsed time in minutes.
            elapsed = format_time(time.time() - t0)
            
            # Report progress.
            print('  Batch {:>5,}  of  {:>5,}.    Elapsed: {:}.'.format(step, len(train_dataloader), elapsed))

        # Unpack this training batch from our dataloader. 
        #
        # As we unpack the batch, we'll also copy each tensor to the GPU using the 
        # `to` method.
        #
        # `batch` contains three pytorch tensors:
        #   [0]: input ids 
        #   [1]: attention masks
        #   [2]: labels 
        b_input_ids = batch[0].to(device)
        b_input_mask = batch[1].to(device)
        b_labels = batch[2].to(device)

        # Always clear any previously calculated gradients before performing a
        # backward pass. PyTorch doesn't do this automatically because 
        # accumulating the gradients is "convenient while training RNNs". 
        # (source: https://stackoverflow.com/questions/48001598/why-do-we-need-to-call-zero-grad-in-pytorch)
        model.zero_grad()        

        # Perform a forward pass (evaluate the model on this training batch).
        # The documentation for this `model` function is here: 
        # https://huggingface.co/transformers/v2.2.0/model_doc/bert.html#transformers.BertForSequenceClassification
        # It returns different numbers of parameters depending on what arguments
        # arge given and what flags are set. For our useage here, it returns
        # the loss (because we provided labels) and the "logits"--the model
        # outputs prior to activation.
        loss, logits = model(b_input_ids, 
                             token_type_ids=None, 
                             attention_mask=b_input_mask, 
                             labels=b_labels)

        # Accumulate the training loss over all of the batches so that we can
        # calculate the average loss at the end. `loss` is a Tensor containing a
        # single value; the `.item()` function just returns the Python value 
        # from the tensor.
        total_train_loss += loss.item()

        # Perform a backward pass to calculate the gradients.
        loss.backward()

        # Clip the norm of the gradients to 1.0.
        # This is to help prevent the "exploding gradients" problem.
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)

        # Update parameters and take a step using the computed gradient.
        # The optimizer dictates the "update rule"--how the parameters are
        # modified based on their gradients, the learning rate, etc.
        optimizer.step()

        # Update the learning rate.
        scheduler.step()

    # Calculate the average loss over all of the batches.
    avg_train_loss = total_train_loss / len(train_dataloader)            
    
    # Measure how long this epoch took.
    training_time = format_time(time.time() - t0)

    print("")
    print("  Average training loss: {0:.2f}".format(avg_train_loss))
    print("  Training epcoh took: {:}".format(training_time))
        
    # ========================================
    #               Validation
    # ========================================
    # After the completion of each training epoch, measure our performance on
    # our validation set.

    print("")
    print("Running Validation...")

    t0 = time.time()

    # Put the model in evaluation mode--the dropout layers behave differently
    # during evaluation.
    model.eval()

    # Tracking variables 
    total_eval_accuracy = 0
    total_eval_loss = 0
    nb_eval_steps = 0

    # Evaluate data for one epoch
    for batch in validation_dataloader:
        
        # Unpack this training batch from our dataloader. 
        #
        # As we unpack the batch, we'll also copy each tensor to the GPU using 
        # the `to` method.
        #
        # `batch` contains three pytorch tensors:
        #   [0]: input ids 
        #   [1]: attention masks
        #   [2]: labels 
        b_input_ids = batch[0].to(device)
        b_input_mask = batch[1].to(device)
        b_labels = batch[2].to(device)
        
        # Tell pytorch not to bother with constructing the compute graph during
        # the forward pass, since this is only needed for backprop (training).
        with torch.no_grad():        

            # Forward pass, calculate logit predictions.
            # token_type_ids is the same as the "segment ids", which 
            # differentiates sentence 1 and 2 in 2-sentence tasks.
            # The documentation for this `model` function is here: 
            # https://huggingface.co/transformers/v2.2.0/model_doc/bert.html#transformers.BertForSequenceClassification
            # Get the "logits" output by the model. The "logits" are the output
            # values prior to applying an activation function like the softmax.
            (loss, logits) = model(b_input_ids, 
                                   token_type_ids=None, 
                                   attention_mask=b_input_mask,
                                   labels=b_labels)
            
        # Accumulate the validation loss.
        total_eval_loss += loss.item()

        # Move logits and labels to CPU
        logits = logits.detach().cpu().numpy()
        label_ids = b_labels.to('cpu').numpy()

        # Calculate the accuracy for this batch of test sentences, and
        # accumulate it over all batches.
        total_eval_accuracy += flat_accuracy(logits, label_ids)
        

    # Report the final accuracy for this validation run.
    avg_val_accuracy = total_eval_accuracy / len(validation_dataloader)
    print("  Accuracy: {0:.2f}".format(avg_val_accuracy))

    # Calculate the average loss over all of the batches.
    avg_val_loss = total_eval_loss / len(validation_dataloader)
    
    # Measure how long the validation run took.
    validation_time = format_time(time.time() - t0)
    
    print("  Validation Loss: {0:.2f}".format(avg_val_loss))
    print("  Validation took: {:}".format(validation_time))

    # Record all statistics from this epoch.
    training_stats.append(
        {
            'epoch': epoch_i + 1,
            'Training Loss': avg_train_loss,
            'Valid. Loss': avg_val_loss,
            'Valid. Accur.': avg_val_accuracy,
            'Training Time': training_time,
            'Validation Time': validation_time
        }
    )

print("")
print("Training complete!")

print("Total training took {:} (h:mm:ss)".format(format_time(time.time()-total_t0)))


======== Epoch 1 / 2 ========
Training...
  Batch    40  of  1,769.    Elapsed: 0:00:51.
  Batch    80  of  1,769.    Elapsed: 0:01:41.
  Batch   120  of  1,769.    Elapsed: 0:02:31.
  Batch   160  of  1,769.    Elapsed: 0:03:21.
  Batch   200  of  1,769.    Elapsed: 0:04:11.
  Batch   240  of  1,769.    Elapsed: 0:05:01.
  Batch   280  of  1,769.    Elapsed: 0:05:51.
  Batch   320  of  1,769.    Elapsed: 0:06:41.
  Batch   360  of  1,769.    Elapsed: 0:07:31.
  Batch   400  of  1,769.    Elapsed: 0:08:22.
  Batch   440  of  1,769.    Elapsed: 0:09:12.
  Batch   480  of  1,769.    Elapsed: 0:10:02.
  Batch   520  of  1,769.    Elapsed: 0:10:52.
  Batch   560  of  1,769.    Elapsed: 0:11:42.
  Batch   600  of  1,769.    Elapsed: 0:12:32.
  Batch   640  of  1,769.    Elapsed: 0:13:22.
  Batch   680  of  1,769.    Elapsed: 0:14:12.
  Batch   720  of  1,769.    Elapsed: 0:15:02.
  Batch   760  of  1,769.    Elapsed: 0:15:52.
  Batch   800  of  1,769.    Elapsed: 0:16:42.
  Batch   840  of

In [ ]:
import pandas as pd

# Display floats with two decimal places.
pd.set_option('precision', 2)

# Create a DataFrame from our training statistics.
df_stats = pd.DataFrame(data=training_stats)

# Use the 'epoch' as the row index.
df_stats = df_stats.set_index('epoch')

# A hack to force the column headers to wrap.
#df = df.style.set_table_styles([dict(selector="th",props=[('max-width', '70px')])])

# Display the table.
df_stats

,Training Loss,Valid. Loss,Valid. Accur.,Training Time,Validation Time
epoch,,,,,
1,1.21,1.02,0.65,0:36:55,0:05:46
2,0.83,0.99,0.66,0:36:54,0:05:47


In [ ]:
import os

# Saving best-practices: if you use defaults names for the model, you can reload it using from_pretrained()

output_dir = './model_save/'

# Create output directory if needed
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

print("Saving model to %s" % output_dir)

# Save a trained model, configuration and tokenizer using `save_pretrained()`.
# They can then be reloaded using `from_pretrained()`
model_to_save = model.module if hasattr(model, 'module') else model  # Take care of distributed/parallel training
model_to_save.save_pretrained(output_dir)
tokenizer.save_pretrained(output_dir)

# Good practice: save your training arguments together with the trained model
# torch.save(args, os.path.join(output_dir, 'training_args.bin'))

Saving model to ./model_save/


('./model_save/vocab.txt',
 './model_save/special_tokens_map.json',
 './model_save/added_tokens.json')

In [ ]:
!ls -l --block-size=K ./model_save/

total 428016K
-rw-r--r-- 1 root root      1K Aug 18 06:02 config.json
-rw-r--r-- 1 root root 427773K Aug 18 06:02 pytorch_model.bin
-rw-r--r-- 1 root root      1K Aug 18 06:02 special_tokens_map.json
-rw-r--r-- 1 root root      1K Aug 18 06:02 tokenizer_config.json
-rw-r--r-- 1 root root    227K Aug 18 06:02 vocab.txt


In [ ]:
# Mount Google Drive to this Notebook instance.
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [ ]:
# Copy the model files to a directory in your Google Drive.
!cp -r ./model_save/ "./drive/My Drive/FIT3161/FYP/"

In [ ]:
# Load a trained model and vocabulary that you have fine-tuned
model = model_save.from_pretrained(output_dir)
tokenizer = tokenizer_class.from_pretrained(output_dir)

# Copy the model to the GPU.
model.to(device)

NameError: ignored

In [ ]:
review_text = "I think this isn't going to work."
encoded_review = tokenizer.encode_plus(
                        review_text,                      # Sentence to encode.
                        add_special_tokens = True, # Add '[CLS]' and '[SEP]'
                        max_length = 64,           # Pad & truncate all sentences.
                        pad_to_max_length = True,
                        return_attention_mask = True,   # Construct attn. masks.
                        return_tensors = 'pt',     # Return pytorch tensors.
                        truncation = True,
                   )
input_ids = encoded_review['input_ids'].to(device)
attention_mask = encoded_review['attention_mask'].to(device)

output = model(input_ids,attention_mask)
print(output)
_,prediction = torch.max(output[0], dim=1)
print(f'Review text: {review_text}')
class_names = ["joy","sadness","disgust","fear","surprise","anger"]
print(f'Sentiment: {class_names[prediction]}')

(tensor([[-0.8971, -3.1025,  1.0751,  0.2081,  0.6861,  2.4460, -1.8378, -0.5258,
          1.1918]], device='cuda:0', grad_fn=<AddmmBackward>),)
Review text: I think this isn't going to work.
Sentiment: anger


In [ ]:
# filtered.loc[filtered["anger"] > 0, 'label'] = 5
# filtered.loc[filtered["fear"] > 0, 'label'] = 3
# filtered.loc[filtered["joy"] > 0, 'label'] = 0
# filtered.loc[filtered["sadness"] > 0, 'label'] = 1
# filtered.loc[filtered["surprise"] > 0, 'label'] = 4
# filtered.loc[filtered["disgust"] > 0, 'label'] = 2

# for i in range(len(filtered)):
#   if filtered[3] > 0:  # anger
#     filtered[2] = 6
#   elif filtered[4] > 0:  # fear
#     filtered[2] = 4
#   elif filtered[5] > 0:  # joy
#     filtered[2] = 1
#   elif filtered[7] > 0:  # sadness
#     filtered[2] = 2
#   elif filtered[8] > 0:  # surprise
#     filtered[2] = 5
#   elif filtered[10] > 0: # disgust
#     filtered[2] = 3